In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import applications
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.optimizers import RMSprop
from keras import backend as K
import numpy as np
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.models import load_model

In [17]:
#parameters
classes = 12
image_size = 1000
batch_size= 6
train_total = 3820
validation_total = 930

In [18]:
model_mobilenet = MobileNet(include_top=False, weights = None,input_shape=(image_size,image_size,3), alpha=0.5)

In [19]:
model_mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [20]:
x = model_mobilenet.output
x = MaxPooling2D((4,4),strides=(4,4))(x)
x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [21]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 1000, 1000, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 500, 500, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 500, 500, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 500, 500, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 500, 500, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 500, 500, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 500, 500, 16)      0         
__________

In [23]:
model_mobilenet_2 = MobileNet(include_top=False, weights = None,input_shape=(1500,1500,3), alpha=0.5)
x = model_mobilenet_2.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [24]:
model_2.load_weights('mobilenet_from_scratch_image_1500_local_split.hdf5')

In [25]:
for layer1,layer2 in zip(model_mobilenet.layers,model_mobilenet_2.layers):
    layer1.set_weights(layer2.get_weights())

In [26]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 45,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [27]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [28]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [29]:
callbacks = [ModelCheckpoint(filepath='mobilenet_fixed_bottom_image_1500_local_split.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=20, verbose=1),
CSVLogger('./12-metrics_fixed_bottom.csv')]

In [ ]:
history = model.fit_generator(
                    train_generator,
                    steps_per_epoch = int(np.ceil(train_total/batch_size)),
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps= int(np.ceil(validation_total/batch_size)),
                    verbose=2,
                    callbacks=callbacks,
                    initial_epoch=24)

Epoch 25/100


In [30]:
model.load_weights('mobilenet_fixed_bottom_image_1500_local_split.hdf5')

In [31]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.12590394568885174, 0.97204300818904754]

In [17]:
# validation predictions

In [32]:
mobilenet_1500_local_split.hdf5validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical',
                        shuffle=False)

Found 930 images belonging to 12 classes.


NameError: name 'mobilenet_1500_local_split' is not defined

In [33]:
predictions_valid = model.predict_generator(validation_generator,int(np.ceil(validation_total/batch_size)))

In [34]:
predictions_valid = np.argmax(predictions_valid, axis=1)


In [35]:
true_positive = np.zeros(12)
false_positive = np.zeros(12)
true_negative = np.zeros(12)
false_negative = np.zeros(12)
for i in range(len(predictions_valid)):
    if(predictions_valid[i]==validation_generator.classes[i]):
        true_positive[predictions_valid[i]] = true_positive[predictions_valid[i]] + 1
    else:
        false_positive[predictions_valid[i]] = false_positive[predictions_valid[i]] + 1
        false_negative[validation_generator.classes[i]] = false_negative[validation_generator.classes[i]] +1

In [36]:
print('true_positive')
print(true_positive)
print('false_positive')
print(false_positive)
print('false_negative')
print(false_negative)
print('Total')
print(true_positive+false_negative)

true_positive
[  38.   94.   63.  107.   43.   85.  128.   36.  106.   41.   95.   68.]
false_positive
[ 4.  1.  0.  4.  2.  1.  9.  3.  1.  0.  0.  1.]
false_negative
[ 9.  0.  2.  1.  2.  4.  3.  0.  1.  1.  0.  3.]
Total
[  47.   94.   65.  108.   45.   89.  131.   36.  107.   42.   95.   71.]


In [112]:
# test submission

In [16]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [17]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [18]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_size_increase_1500_local_split_only.csv', encoding="utf8", index=False)

In [19]:
from IPython.display import FileLink
FileLink('submission_mobilenet_size_increase_1500_local_split_only.csv')

/home/sainath/plant_seeding/submission_mobilenet_size_increase_1500_local_split_only.csv

In [ ]:
# PB LB: 0.97858